# Figures 4 and 5

In [18]:
import pandas as pd
import numpy as np
import json
from matplotlib import pyplot as plt
import seaborn as sns

In [19]:
with open("LightGBM_results_NO_celltype.json", "r") as data:
    lightgbm_results_wo_celltypes = json.load(data)
    for k, v in lightgbm_results_wo_celltypes.items():
        lightgbm_results_wo_celltypes[k] = np.mean(list(v.values()))
    
with open("0v60.json", "r") as data:
    deepST_results_wo_celltypes_by_gene = json.load(data)
    deepST_results_wo_celltypes = {}
    deepST_results_wo_celltypes["0"] = np.mean([deepST_results_wo_celltypes_by_gene["5"][x]["base"] for x in deepST_results_wo_celltypes_by_gene["5"]])
    for rad in range(5, 51, 5):
        deepST_results_wo_celltypes[str(rad)] = np.mean([deepST_results_wo_celltypes_by_gene[str(rad)][x]["spatial"] for x in deepST_results_wo_celltypes_by_gene[str(rad)]])
    
with open("LightGBM_results.json", "r") as data:
    lightgbm_results_with_celltypes = json.load(data)
    for k, v in lightgbm_results_with_celltypes.items():
        lightgbm_results_with_celltypes[k] = np.mean(list(v.values()))

with open("0v60_w_celltypes.json", "r") as data:
    deepST_results_w_celltypes_by_gene = json.load(data)
    deepST_results_w_celltypes = {}
    deepST_results_w_celltypes["0"] = np.mean([deepST_results_w_celltypes_by_gene[str(rad)][x]["base"] for x in deepST_results_w_celltypes_by_gene["5"]])
    for rad in range(5, 51, 5):
        deepST_results_w_celltypes[str(rad)] = np.mean([deepST_results_w_celltypes_by_gene[str(rad)][x]["spatial"] for x in deepST_results_w_celltypes_by_gene[str(rad)]])

In [ ]:
# Create DataFrames for results without and with cell types
lightgbm_results_wo_celltypes_df = pd.DataFrame(lightgbm_results_wo_celltypes.items(), columns=['Radius', 'w/o Celltypes Loss'])
lightgbm_results_with_celltypes_df = pd.DataFrame(lightgbm_results_with_celltypes.items(), columns=['Radius', 'w/ Celltypes Loss'])
deepST_results_wo_celltypes_df = pd.DataFrame(deepST_results_wo_celltypes.items(), columns=['Radius', 'w/o Celltypes Loss'])
deepST_results_with_celltypes_df = pd.DataFrame(deepST_results_w_celltypes.items(), columns=['Radius', 'w/ Celltypes Loss'])

# Combine DataFrames for plotting
no_celltypes_df = pd.concat((lightgbm_results_wo_celltypes_df, deepST_results_wo_celltypes_df), axis=0)
celltypes_df = pd.concat((lightgbm_results_with_celltypes_df, deepST_results_with_celltypes_df), axis=0)

# Function to check if a value is an integer
def is_int(n):
    try:
        int(n)  # Try to convert to int
        return True
    except ValueError:
        return False  # Return False if conversion fails

# Manually process no_celltypes_df
no_celltypes_df["Method"] = no_celltypes_df["Radius"].apply(lambda x: x.split(' ')[0] if not is_int(x.split(' ')[0]) else "DeepST")
no_celltypes_df["Radius"] = no_celltypes_df["Radius"].apply(lambda x: x.split(' ')[-1])
no_celltypes_df = no_celltypes_df[no_celltypes_df['Radius'].astype(int) <= 50]

# Manually process celltypes_df
celltypes_df["Method"] = celltypes_df["Radius"].apply(lambda x: x.split(' ')[0] if not is_int(x.split(' ')[0]) else "DeepST")
celltypes_df["Radius"] = celltypes_df["Radius"].apply(lambda x: x.split(' ')[-1])
celltypes_df = celltypes_df[celltypes_df['Radius'].astype(int) <= 50]

# Filter out 'OLS' method
no_celltypes_df = no_celltypes_df[no_celltypes_df["Method"] != "OLS"]

# Set the plot style
sns.set_theme("talk")
sns.set_style("whitegrid")

# Create the line plot
sns.lineplot(data=no_celltypes_df, x="Radius", y="w/o Celltypes Loss", marker='o', hue="Method", style="Method", markers=["o", "s", "D", "^", "v"])

# Adjust legend and save the figure
plt.legend(bbox_to_anchor=(1, 1))
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.savefig('real_data_wo_celltypes.png', bbox_inches="tight")

In [ ]:
celltypes_df = celltypes_df[celltypes_df["Method"] != "OLS"]

sns.set_theme("talk")
sns.set_style("whitegrid")
# Create a line plot for each model with different markers
sns.lineplot(data=celltypes_df, x="Radius", y="w/ Celltypes Loss", marker='o', hue="Method", style="Method", markers=["o", "s", "D", "^", "v"])

# Add a horizontal dashed line at y=0.3595
plt.axhline(y=0.3595, color='purple', linestyle='--', label='MESSI')

ax = plt.gcf().gca()

plt.legend(bbox_to_anchor=(1, 1))
# plt.legend(labels=['LightGBM Test Loss', 'deepST Test Loss'], bbox_to_anchor=(1, 1))

plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.savefig('real_data_WITH_celltypes.png', bbox_inches="tight")

# Figure 6

In [22]:
response_indexes = [0,2,3,4,5,6,7,10,19,20,21,22,23,24,25,26,27,28,32,34,35,37,38,39,40,41,42,43,44,52,53,54,55,58,63,64,66,67,69,71,73,74,75,76,77,78,79,80,85,86,87,88,93,94,96,97,99,102,103,104,106,110,112,113,114,116,118,119,120,121,122,123,124,125,126,129,130,131,133,134,141,142,147,151]

In [23]:
data = pd.read_csv("../data/raw/merfish.csv")
data = data.drop(["Fos", "Blank_1", "Blank_2", "Blank_3", "Blank_4", "Blank_5"], axis=1)
data = data.iloc[:, 9:]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import json

with open("0v60.json", "r") as file:
    loss_dict = json.load(file)

radius_values = list(range(0, 26, 5))
genes = [data.columns[i] for i in response_indexes]

loss_array = np.array([
    [-loss_dict[str(rad)][gene]["percent_diff"] if rad != 0 else 0 for rad in radius_values] for gene in genes
])

# regenerate the loss array with gene filter
THRESHOLD_ABS = 20
THRESHOLD_MIN = 0
genes = [genes[i] for i in range(len(genes)) if ( (sum([np.abs(x) for x in loss_array[i]]) > THRESHOLD_ABS) or sum(loss_array[i]) < THRESHOLD_MIN )]
loss_array = np.array([
    [-loss_dict[str(rad)][gene]["percent_diff"] if rad != 0 else 0 for rad in radius_values] for gene in genes
])

# Set the figure size based on the dimensions of your data
cell_width = 2.5
cell_height = 0.5
fig, ax = plt.subplots(figsize=(cell_width*len(radius_values), cell_height*len(genes)))

im = ax.imshow(loss_array, cmap='seismic', vmin=-np.max(np.abs(loss_array)), vmax=np.max(np.abs(loss_array)))

# Add a colorbar for reference with adjusted fraction to make it thinner
cbar = ax.figure.colorbar(im, ax=ax, fraction=0.05)
cbar.ax.set_ylabel("% Loss Reduction", fontsize=20, rotation=-90, va="bottom")

ax.set_xticks(np.arange(len(radius_values)))
ax.set_xticklabels(list(radius_values))
ax.set_yticks(np.arange(len(genes)))
ax.set_yticklabels(genes)

# Adjust font size
ax.tick_params(axis='both', which='major', labelsize=10)

plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")

# Dynamically adjust text color based on cell value and format the number to 2 decimal places
for i in range(len(genes)):
    for j in range(len(radius_values)):
        if loss_array[i, j] < 0:  # If value is negative, use white text
            text_color = 'red'
        else:  # Otherwise, use black text
            text_color = 'green'
        text = ax.text(j, i, f"{loss_array[i, j]:.2f}",
                       ha="center", va="center", color=text_color, fontsize=10)

# ax.set_title("Predictions Across $r$ Values", fontsize=25)
# fig.tight_layout()
plt.savefig("spatial_vertical.png", dpi=600)
plt.show()

# Figure 7

In [25]:
with open("0v60.json", "r") as f:
    loss_dict = json.load(f)

In [26]:
percent_differences = [-loss_dict["25"][x]["percent_diff"] for x in loss_dict["25"]]

In [ ]:
plt.hist(percent_differences, bins=len(loss_dict["25"]))
plt.title("% Loss reduction in spatial model relative to baseline.")
plt.xlabel("Percent Reduction in Test Loss")
plt.annotate("Nnat", xy=(39.5866-2, 1.25))
plt.annotate("Mbp", xy=(24.8099-1.3, 1.25))
plt.savefig("0VSspatial.png", dpi=300)
plt.show()

# Figure 8

In [28]:
with open('deepST_synthetic_results.json', 'r') as deepST:
    test_loss_rad_dict = json.load(deepST)

In [29]:
with open('LightGBM_synthetic_results.json', 'r') as linear:
    linear_models_dict = json.load(linear)

# Figure 9

In [ ]:
# create dataframe
import pandas as pd

# linear models + XGBoost
data = pd.DataFrame(columns=['Model', 'Radius', 'Experiment #', 'L2 Loss'])
for k,v in linear_models_dict.items():
    k = k.split()
    entry = pd.DataFrame.from_dict({
        'Model': [k[0]],
        'Radius': [int(k[1])],
        'Experiment #': [k[2]],
        'L2 Loss': [v]
    })
    data = pd.concat([data, entry], ignore_index=True)

print(data)


for k,v in test_loss_rad_dict.items():
    try:
        k = eval(k)
        entry = pd.DataFrame.from_dict({
            'Model': 'deepST',
            'Radius': [int(k[0])],
            'Experiment #': [str(k[1])],
            'L2 Loss': [min(v.values())]
        })
        data = pd.concat([data, entry], ignore_index=True)
    #nonlinear case
    except (AttributeError, ValueError):
        entry = pd.DataFrame.from_dict({
            'Model': 'deepST',
            'Radius': [int(k[0])],
            'Experiment #': [str(k[1])],
            'L2 Loss': [v]
        })
        data = pd.concat([data, entry], ignore_index=True)
        

data[data["Model"] == "deepST"]

In [ ]:
EXPERIMENT_NUM = "1"

# Set styles before plotting
plt.rcParams['font.family'] = 'DejaVu Sans'
sns.set(rc={"lines.linewidth": 0.7})

# Create pointplot for all but the specific model
model_to_highlight = "LightGBM"
new_data = data[(data['Experiment #'] == EXPERIMENT_NUM) & ((data["Model"] != "OLS") & (data["Model"] != "Ridge") & (data['Model'] != model_to_highlight))]
synth_results = sns.pointplot(x='Radius', y='L2 Loss', hue='Model', data=new_data)

# Set the y-limits
synth_results.set_ylim(bottom=0, top=1.25*max(new_data["L2 Loss"]))

# Label and title configurations
synth_results.set_xlabel("Radius", fontsize=20)
synth_results.set_ylabel("MSE Loss", fontsize=20)

# Create pointplot just for the specific model to highlight
highlight_data = data[(data['Model'] == model_to_highlight) & (data['Experiment #'] == EXPERIMENT_NUM)]
sns.pointplot(x='Radius', y='L2 Loss', hue='Model', data=highlight_data, markers=["D"], linestyles=["--"], color="red", dodge=0.2)  # Use different markers and linestyles for emphasis if you like

# Save the plot
plt.savefig(f'synthetic{EXPERIMENT_NUM}.png', dpi=300, bbox_inches='tight')

In [ ]:
EXPERIMENT_NUM = "Nonlinear"

# Set styles before plotting
plt.rcParams['font.family'] = 'DejaVu Sans'
sns.set(rc={"lines.linewidth": 0.7})

# Create pointplot for all but the specific model
model_to_highlight = "LightGBM"
new_data = data[(data['Experiment #'] == EXPERIMENT_NUM) & ((data["Model"] != "OLS") & (data["Model"] != "Ridge") & (data['Model'] != model_to_highlight))]
synth_results = sns.pointplot(x='Radius', y='L2 Loss', hue='Model', data=new_data)

# Set the y-limits
synth_results.set_ylim(bottom=0, top=1.25*max(new_data["L2 Loss"]))

# Label and title configurations
synth_results.set_xlabel("Radius", fontsize=20)
synth_results.set_ylabel("MSE Loss", fontsize=20)

# Create pointplot just for the specific model to highlight
highlight_data = data[(data['Model'] == model_to_highlight) & (data['Experiment #'] == EXPERIMENT_NUM)]
sns.pointplot(x='Radius', y='L2 Loss', hue='Model', data=highlight_data, markers=["D"], linestyles=["--"], color="red", dodge=0.2)  # Use different markers and linestyles for emphasis if you like

# Save the plot
plt.savefig(f'synthetic{EXPERIMENT_NUM}.png', dpi=300, bbox_inches='tight')